This notebook converts MOM model data into a readable format for the [interactivate validator](http://rshiny.io-warnemuende.de/radtke/validator/)

> see https://youtu.be/S-UkKqyolJc for a full tutorial

In [1]:
from netCDF4 import Dataset
import os, glob, re
import xarray as xr

In [2]:
BEGIN               = 1961
END                 = 2010
STATION_SYNTAX      = "rregion_"
variables_in_model  = ["salt"]
validator_names     = ["MODEL_SALIN"]

DIR             = "/silor/boergel/paper/bmip/data/MOM/balt-3nm-skag-v02-r04_"

if os.path.isdir("for_validator") == True:
    print("Validator folder already exsists")
else:
    os.mkdir('for_validator')

Validator folder already exsists


In [3]:
for i in range(BEGIN, END):
    station_list = [x for x in glob.glob(DIR + str(i) +'/*'+ STATION_SYNTAX +'*')]
    for station in station_list:
        if "transect" in station:
            continue
        STATION_NAME = station.split(STATION_SYNTAX)[1].split(".nc.")[0]
        try:
            ds = xr.open_dataset(station)
            xt_ocean_pattern = re.compile("xt_ocean*")
            xt_ocean = [x for x in ds.dims if xt_ocean_pattern.match(x)][0]
            yt_ocean_pattern = re.compile("yt_ocean*")
            yt_ocean = [x for x in ds.dims if yt_ocean_pattern.match(x)][0]
            depth_pattern = re.compile("st_ocean_*")
            depth_name = [x for x in ds.dims if depth_pattern.match(x)][0]


            ds = ds.rename({depth_name:"z"})
            ds = ds[variables_in_model].squeeze()
            for var in range(len(variables_in_model)):
                ds = ds.rename({variables_in_model[var]:validator_names[var]})
            ds.to_netcdf("for_validator/" + STATION_NAME + "_" + str(i) + ".nc")
        except:
#                 print("{} did not work".format(STATION_NAME))
                continue

In [4]:
for station in station_list:
    try:
        if "transect" in station:
            continue
        STATION_NAME = station.split(STATION_SYNTAX)[1].split(".nc.")[0]
        print(STATION_NAME)
        ds = xr.open_mfdataset("for_validator/" + STATION_NAME+"_*.nc")
        ds.to_netcdf("for_validator/" + STATION_NAME + ".nc")
    except Exception as e:
        print(STATION_NAME, e)

SEGotlandBasin
ArkonaBY2
GulfFinlandLL7
FehmarnBelt
GotlandDeepBY15
BothnianSeaSR5
GulfofFinlandF1
LandskronaW
AnholtE
GdanskDeep
GreatBelt
BornholmDeepBY5
BothnianBayF9
GulfofRiga
LandsortDeepBY31
LandskronaW


In [5]:
for station in station_list:
    if "transect" in station:
        continue
    STATION_NAME = station.split(STATION_SYNTAX)[1].split(".nc.")[0]
    for file in glob.glob("for_validator/" + STATION_NAME+"_*.nc"):
        os.remove(file)
for file in glob.glob("for_validator/trans*"):
    os.remove(file)

In [6]:
STATION_NAMES = [station.split("for_validator/")[1] for station in glob.glob("for_validator/*")]
STATION_NAMES_VALIDATOR = ["AnholtE","by2", "by5", "f9", "sr5", "fehmarn", "gdansk", "by15", "greatbelt", "ll7", "f1", "gulfofriga", "by31", "segotlandbasin"]

In [7]:
STATION_NAMES = sorted(STATION_NAMES)

In [8]:
for count, station in enumerate(STATION_NAMES):
    os.rename("for_validator/" + station, "for_validator/" + STATION_NAMES_VALIDATOR[count] + ".nc")

IndexError: list index out of range